In [1]:
# import packages
import torch
import torch.nn as nn 
import torch.nn.functional as F 
import pandas as pd 
from tqdm import tqdm
from torchtext.legacy import data
from torchtext.vocab import Vectors
from torchtext.vocab import GloVe
import time

In [2]:
# preprocessing

from nltk import word_tokenize

text = data.Field(sequential = True, lower = True, tokenize = word_tokenize)
term = data.Field(sequential = False, lower = True)
polarity = data.Field(sequential = False)

In [3]:
train, val = data.TabularDataset.splits(path=r'data/',
                                        skip_header=True,
                                        train='rest_train.csv',
                                        validation='rest_test.csv',
                                        format='csv',
                                        fields=[('text', text),
                                                ('term', term),
                                                ('polarity', polarity)])

In [5]:
vectors = Vectors(name='data/glove.6B.300d.txt')

100%|█████████▉| 399999/400000 [00:36<00:00, 10848.35it/s]


In [6]:
text.build_vocab(train, val, vectors=vectors)
term.build_vocab(train, val, vectors=vectors)
polarity.build_vocab(train, val)

text_vocab_size = len(text.vocab)
term_vocab_size = len(term.vocab)
text_vector=text.vocab.vectors
term_vector=term.vocab.vectors

In [7]:
batch_size=512
train_iter, val_iter = data.Iterator.splits(
            (train, val),
            sort_key=lambda x: len(x.text),
            batch_sizes=(batch_size, len(val)), # batch_size only for training
    )   

In [19]:
class ATAE_LSTM(nn.Module):
    def __init__(self, embedding_dim, num_hiddens, num_layers):
        super(ATAE_LSTM, self).__init__()
        self.text_embeddings = nn.Embedding(text_vocab_size, embedding_dim)
        self.term_embeddings = nn.Embedding(term_vocab_size, embedding_dim)
        self.text_embeddings = nn.Embedding.from_pretrained(text_vector,
                                                            freeze=False)
        self.term_embeddings = nn.Embedding.from_pretrained(term_vector,
                                                              freeze=False)
        self.lstm = nn.LSTM(input_size=2 * embedding_dim,
                            hidden_size=num_hiddens,
                            num_layers=num_layers,
                            batch_first=True,
                            bidirectional=True)
        self.wh = nn.Parameter(torch.Tensor(num_hiddens * 2, num_hiddens * 2))
        self.wv = nn.Parameter(torch.Tensor(embedding_dim, embedding_dim))
        self.omega = nn.Parameter(
            torch.Tensor(1, embedding_dim*2))
        self.wp = nn.Parameter(torch.Tensor(num_hiddens * 2, num_hiddens * 2))
        self.wx = nn.Parameter(torch.Tensor(num_hiddens * 2, num_hiddens * 2))
        self.ws = nn.Parameter(torch.Tensor(4, num_hiddens * 2))
        nn.init.uniform_(self.wh, -0.1, 0.1)
        nn.init.uniform_(self.wv, -0.1, 0.1)
        nn.init.uniform_(self.omega, -0.1, 0.1)
        nn.init.uniform_(self.wp, -0.1, 0.1)
        nn.init.uniform_(self.wx, -0.1, 0.1)
        nn.init.uniform_(self.ws, -0.1, 0.1)
        self.bs = nn.Parameter(torch.zeros((4, 1)))

    def forward(self, text, term):
        seq_len = len(text.t())
        e1 = self.text_embeddings(text)
        # e1 shape(batch_size,seq_len, embedding_dim)
        e2 = self.term_embeddings(term).expand(e1.size())

        wv = torch.cat((e1, e2), dim=2)
        # e.g.
        # wv torch.Size([batch_size,seq_len,2*embedding_dim])

        out, (h, c) = self.lstm(wv)  # output, (h, c)
        # out shape(batch_size,seq_len, 2 * num_hiddens)
        # h shape(num_layers * num_directions, batch_size, 2*num_hiddens)

        H = out.permute(0, 2, 1)
        # H shape(batch_size,2 * num_hiddens,seq_len)
        #print(H.shape)
        #print(self.wh.shape)

        Wh_H = torch.matmul(self.wh, H)
        # wh shape(2*num_hiddens, 2*num_hiddens)
        # wh_H shape(batch_size, 2*num_hiddens, seq_len)
        # print('Wh_H: ', Wh_H.shape)

        Wv_Va_eN = torch.matmul(
            self.wv,
            self.term_embeddings(term).permute(0, 2, 1).expand(-1, embedding_dim, seq_len))
        # Wv shape(seq_len, seq_len) embedding_dim=2*num_hiddens
        # Wv_Va_eN shape(batch_size, embedding_dim, seq_len)
        # print('Wv_Va_eN: ', Wv_Va_eN.shape)

        vh = torch.cat((Wh_H, Wv_Va_eN), dim=1)
        # vh shape(batch_size, embedding_dim + 2*num_hiddens, seq_len)
        # print('vh: ', vh.shape)

        M = torch.tanh(vh)
        # M shape(batch_size, embedding_dim + 2*num_hiddens, seq_len)
        #print('M: ', M.shape)

        alpha = F.softmax(torch.matmul(self.omega, M),dim=2)
        # omega shape(1, embedding_dim + 2*num_hiddens))
        # alpha shape(batch_size, 1, seq_len)
        #print('alpha: ', alpha.shape)

        r = torch.matmul(H, alpha.permute(0, 2, 1))
        # H shape(batch_size,2 * num_hiddens,seq_len)
        # r shape(batch_size,2*num_hiddens,1)
        #print('r: ', r.shape)

        h_star = torch.tanh(
            torch.matmul(self.wp, r) +
            torch.matmul(self.wx, torch.unsqueeze(H[:, :, -1], 2)))
        # h_star shape(batch_size,2 * num_hiddens,1)
        #print('h_star: ', h_star.shape)

        y = torch.matmul(self.ws, h_star) + self.bs  
        # y shape(batch_size,4,1)
        
        y = y.reshape([-1, 4])
        # y shape(batch_size,4)
        # ws shape(4, 2*num_hiddens)
        #print('y: ', y.shape)

        return y

In [ ]:
class ATAE_GRU(nn.Module):
    def __init__(self, embedding_dim, num_hiddens, num_layers):
        super(ATAE_LSTM, self).__init__()
        self.text_embeddings = nn.Embedding(text_vocab_size, embedding_dim)
        self.term_embeddings = nn.Embedding(term_vocab_size, embedding_dim)
        self.text_embeddings = nn.Embedding.from_pretrained(text_vector,
                                                            freeze=False)
        self.term_embeddings = nn.Embedding.from_pretrained(term_vector,
                                                              freeze=False)
        self.gru = nn.GRU(input_size=2 * embedding_dim,
                            hidden_size=num_hiddens,
                            num_layers=num_layers,
                            batch_first=True,
                            bidirectional=True)
        self.wh = nn.Parameter(torch.Tensor(num_hiddens * 2, num_hiddens * 2))
        self.wv = nn.Parameter(torch.Tensor(embedding_dim, embedding_dim))
        self.omega = nn.Parameter(
            torch.Tensor(1, embedding_dim*2))
        self.wp = nn.Parameter(torch.Tensor(num_hiddens * 2, num_hiddens * 2))
        self.wx = nn.Parameter(torch.Tensor(num_hiddens * 2, num_hiddens * 2))
        self.ws = nn.Parameter(torch.Tensor(4, num_hiddens * 2))
        nn.init.uniform_(self.wh, -0.1, 0.1)
        nn.init.uniform_(self.wv, -0.1, 0.1)
        nn.init.uniform_(self.omega, -0.1, 0.1)
        nn.init.uniform_(self.wp, -0.1, 0.1)
        nn.init.uniform_(self.wx, -0.1, 0.1)
        nn.init.uniform_(self.ws, -0.1, 0.1)
        self.bs = nn.Parameter(torch.zeros((4, 1)))

    def forward(self, text, term):
        seq_len = len(text.t())
        e1 = self.text_embeddings(text)
        # e1 shape(batch_size,seq_len, embedding_dim)
        e2 = self.term_embeddings(term).expand(e1.size())

        wv = torch.cat((e1, e2), dim=2)
        # e.g.
        # wv torch.Size([batch_size,seq_len,2*embedding_dim])

        out, (h, c) = self.gru(wv)  # output, (h, c)
        # out shape(batch_size,seq_len, 2 * num_hiddens)
        # h shape(num_layers * num_directions, batch_size, 2*num_hiddens)

        H = out.permute(0, 2, 1)
        # H shape(batch_size,2 * num_hiddens,seq_len)
        #print(H.shape)
        #print(self.wh.shape)

        Wh_H = torch.matmul(self.wh, H)
        # wh shape(2*num_hiddens, 2*num_hiddens)
        # wh_H shape(batch_size, 2*num_hiddens, seq_len)
        # print('Wh_H: ', Wh_H.shape)

        Wv_Va_eN = torch.matmul(
            self.wv,
            self.term_embeddings(term).permute(0, 2, 1).expand(-1, embedding_dim, seq_len))
        # Wv shape(seq_len, seq_len) embedding_dim=2*num_hiddens
        # Wv_Va_eN shape(batch_size, embedding_dim, seq_len)
        # print('Wv_Va_eN: ', Wv_Va_eN.shape)

        vh = torch.cat((Wh_H, Wv_Va_eN), dim=1)
        # vh shape(batch_size, embedding_dim + 2*num_hiddens, seq_len)
        # print('vh: ', vh.shape)

        M = torch.tanh(vh)
        # M shape(batch_size, embedding_dim + 2*num_hiddens, seq_len)
        #print('M: ', M.shape)

        alpha = F.softmax(torch.matmul(self.omega, M),dim=2)
        # omega shape(1, embedding_dim + 2*num_hiddens))
        # alpha shape(batch_size, 1, seq_len)
        #print('alpha: ', alpha.shape)

        r = torch.matmul(H, alpha.permute(0, 2, 1))
        # H shape(batch_size,2 * num_hiddens,seq_len)
        # r shape(batch_size,2*num_hiddens,1)
        #print('r: ', r.shape)

        h_star = torch.tanh(
            torch.matmul(self.wp, r) +
            torch.matmul(self.wx, torch.unsqueeze(H[:, :, -1], 2)))
        # h_star shape(batch_size,2 * num_hiddens,1)
        #print('h_star: ', h_star.shape)

        y = torch.matmul(self.ws, h_star) + self.bs  
        # y shape(batch_size,4,1)
        
        y = y.reshape([-1, 4])
        # y shape(batch_size,4)
        # ws shape(4, 2*num_hiddens)
        #print('y: ', y.shape)

        return y

In [16]:
embedding_dim, num_hiddens, num_layers = 300, 150, 1
net = ATAE_LSTM(embedding_dim, num_hiddens, num_layers).cuda()
print(net)

ATAE_LSTM(
  (text_embeddings): Embedding(4545, 300)
  (term_embeddings): Embedding(1529, 300)
  (lstm): LSTM(600, 150, batch_first=True, bidirectional=True)
)


In [10]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_iter):
            X1, X2, y = batch.text.cuda(), batch.term.cuda(), batch.polarity.cuda()
            X1 = X1.permute(1, 0)
            X2 = X2.unsqueeze(1)
            y.data.sub_(1)  # index start from 0
            if isinstance(net, torch.nn.Module):
                net.eval()  
                acc_sum += (net(X1,
                                X2).argmax(dim=1) == y).float().sum().item()
                net.train()  
            else:
                if ('is_training'
                        in net.__code__.co_varnames): 
                    acc_sum += (net(X1, X2, is_training=False).argmax(
                        dim=1) == y).float().sum().item()
                else:
                    acc_sum += (net(
                        X1, X2).argmax(dim=1) == y).float().sum().item()
            n += y.shape[0]
    return acc_sum / n

In [11]:
def train(train_iter, test_iter, net, loss, optimizer, num_epochs):
    batch_count = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for batch_idx, batch in enumerate(train_iter):
            X1, X2, y = batch.text.cuda(), batch.term.cuda(), batch.polarity.cuda()
            X1 = X1.permute(1, 0).cuda()
            X2 = X2.unsqueeze(1).cuda()
            y.data.sub_(1)  # index start from 0
            y_hat = net(X1,X2)
            l = loss(y_hat, y)

            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print(
            'epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
            % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n,
               test_acc, time.time() - start))

In [17]:
embedding_dim, num_hiddens, num_layers = 300, 150, 1
net = ATAE_LSTM(embedding_dim, num_hiddens, num_layers).cuda()
print(net)

ATAE_LSTM(
  (text_embeddings): Embedding(4545, 300)
  (term_embeddings): Embedding(1529, 300)
  (lstm): LSTM(600, 150, batch_first=True, bidirectional=True)
)


In [18]:
lr, num_epochs = 0.001, 20
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss()
train(train_iter, val_iter, net, loss, optimizer, num_epochs)

Wh_H:  torch.Size([512, 300, 66])
Wv_Va_eN:  torch.Size([512, 300, 66])
vh:  torch.Size([512, 600, 66])
Wh_H:  torch.Size([512, 300, 79])
Wv_Va_eN:  torch.Size([512, 300, 79])
vh:  torch.Size([512, 600, 79])
Wh_H:  torch.Size([512, 300, 79])
Wv_Va_eN:  torch.Size([512, 300, 79])
vh:  torch.Size([512, 600, 79])
Wh_H:  torch.Size([512, 300, 79])
Wv_Va_eN:  torch.Size([512, 300, 79])
vh:  torch.Size([512, 600, 79])
Wh_H:  torch.Size([512, 300, 79])
Wv_Va_eN:  torch.Size([512, 300, 79])
vh:  torch.Size([512, 600, 79])
Wh_H:  torch.Size([512, 300, 79])
Wv_Va_eN:  torch.Size([512, 300, 79])
vh:  torch.Size([512, 600, 79])
Wh_H:  torch.Size([512, 300, 69])
Wv_Va_eN:  torch.Size([512, 300, 69])
vh:  torch.Size([512, 600, 69])
Wh_H:  torch.Size([18, 300, 38])
Wv_Va_eN:  torch.Size([18, 300, 38])
vh:  torch.Size([18, 600, 38])
Wh_H:  torch.Size([1120, 300, 70])
Wv_Va_eN:  torch.Size([1120, 300, 70])
vh:  torch.Size([1120, 600, 70])
epoch 1, loss 1.0363, train acc 0.560, test acc 0.654, time 0.5 

KeyboardInterrupt: 